In [1]:
import os
from sqlalchemy import create_engine, text
import  config as config
import  query as query


In [2]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns

In [3]:
#Conexión BD 
# text(query.sql) -> En la versiones superiores a la 2.0 de  sqlalchemy  se elimina la funcion  Engine.execute , entonces se debe usar TextClause, corresponde con una cadena SQL textual mediante la función sqlalchemy.text. 
_engine = create_engine(f'oracle+cx_oracle://{config.username()}:{config.password()}@{config.dns()}',max_overflow=5_000)
_conexion = _engine.connect()
_alchemy_query = text(query.sql) 

In [4]:
data = pd.read_sql_query(sql=_alchemy_query , con=_conexion) 

## dejo el data frame original
df_data = data

valores = [2018,2019,2021,2022]
df_data = df_data[df_data['anos_ccod'].isin(valores)]

In [5]:
df_data = df_data.drop(['pers_ncorr','emat_tdesc','pers_nrut','rut_alumno_cdv','anos_ccod'],axis=1)

In [6]:
df_data.isnull().sum()

inst_tnemotecnia                    0
sede_tdesc                          0
zona_tdesc                          0
area_tdesc                          0
jorn_tdesc                          0
sexo_tdesc                          0
tcol_tdesc                      10032
cant_dias_matr                      0
parentesco_fiador_new               0
cant_periodos_postulaciones         0
veces_accesos_app                   0
ev_diagnostica_2                  373
ev_diagnostica_4                  284
sin_beneficios                      0
cae                                 0
promedio_asistencia                 0
monto_deuda                         0
promedio_notas                      0
responde_encuesta_an                0
p_asig_riesgo                       0
codigo_asignatura_mat_alg_ss        0
es_desertor                         0
dtype: int64

# Tratamiento de nulos

In [7]:
# tcol_tdesc                            dejo una nueva categoria 
df_data['tcol_tdesc'] = df_data['tcol_tdesc'].fillna('Sin Categoria')

In [8]:
df_data['tcol_tdesc'].value_counts()

Particular Subvencionado                            55900
Municipal                                           26375
Corporación Municipal                               13648
Sin Categoria                                       10032
Corporación de Administración Delegada (DL 3166)     7758
Servicio Local de Educación                          7703
Particular Pagado o no subvencionado                 4577
Name: tcol_tdesc, dtype: int64

In [9]:
## ev_diagnostica_2 lleno con 0 y discretizo la variable
df_data['ev_diagnostica_2'] = df_data['ev_diagnostica_2'].fillna(0)
df_data['ev_diagnostica_4'] = df_data['ev_diagnostica_4'].fillna(0)

# Genero Archivo con informacion de datos

In [10]:
import ydata_profiling
#profile = ydata_profiling.ProfileReport(df_data) # generamos el perfil de datos
#profile.to_file('html/analisis_variables.html') # almacenamos el perfil de datos en un archivo html


ModuleNotFoundError: No module named 'ydata_profiling'

# Discretización variables numericas

In [11]:
from sklearn.preprocessing import KBinsDiscretizer 

In [12]:
#n_bins: número de intervalos a discretizar. Tiene que ser mayor que 2 y por defecto toma el valor de 5.
#encode: Método utilizado para codificar el resultado transformado. 
#    Tiene tres opciones: ‘onehot‘ , ‘onehot-dense‘ y ‘ordinal‘. 
#    La primera de las codificaciones devuelve una matriz dispersa por cada valor, 
#    la segunda codificación devuelve un array y 
#    la última un valor entero. Por defecto toma la codificación onehot.

#strategy: Es la estrategia usada para definir cuántos valores hay en cada intervalo discretizado. 
#    Tiene tres opciones: ‘uniform‘, donde todos los intervalos tienen el mismo ancho, 
#                         ‘quantile‘, donde cada intervalo tiene el mismo número de valores   
#                         ‘kmeans‘, que agrupa los datos de cada intervalo según su centroide más cercano. Por defecto toma el valor de quantile.

In [12]:
df_data['ev_diagnostica_2_new'] = KBinsDiscretizer(n_bins=11, encode='ordinal',
                           strategy = "kmeans").fit_transform(df_data[['ev_diagnostica_2']])

In [13]:
df_data = df_data.drop(['ev_diagnostica_2'] , axis = 1)

In [14]:
df_data['ev_diagnostica_4_new'] = KBinsDiscretizer(n_bins=11, encode='ordinal',
                           strategy = "kmeans").fit_transform(df_data[['ev_diagnostica_4']])

In [15]:
df_data = df_data.drop(['ev_diagnostica_4'] , axis = 1)

In [16]:
df_data['p_asig_riesgo_new'] = KBinsDiscretizer(n_bins=11, encode='ordinal',
                           strategy = "kmeans").fit_transform(df_data[['p_asig_riesgo']])

In [17]:
df_data = df_data.drop(['p_asig_riesgo'] , axis = 1)

In [18]:
#Analizo la variable accesos 
df_data['p_asig_riesgo_new'].value_counts()

0.0     32590
2.0     26551
3.0     15809
10.0    13321
4.0      9567
5.0      7274
6.0      5597
7.0      4857
8.0      4799
1.0      4736
9.0       892
Name: p_asig_riesgo_new, dtype: int64

In [19]:
df_data['veces_accesos_app'].describe()

count    125993.000000
mean         37.855929
std          27.188474
min           0.000000
25%          16.000000
50%          31.000000
75%          54.000000
max         223.000000
Name: veces_accesos_app, dtype: float64

In [20]:
## TRansformo y normalizo veces accesos app y monto deuda
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [21]:
print(scaler.fit(df_data[['veces_accesos_app']]))
StandardScaler()
df_data['veces_accesos_app_new'] = scaler.transform(df_data[['veces_accesos_app']])
df_data = df_data.drop(['veces_accesos_app'], axis = 1)

StandardScaler()


In [22]:
print(scaler.fit(df_data[['monto_deuda']]))
StandardScaler()
df_data['monto_deuda_new'] = scaler.transform(df_data[['monto_deuda']])
df_data = df_data.drop(['monto_deuda'], axis = 1)

StandardScaler()


In [23]:
df_data.columns

Index(['inst_tnemotecnia', 'sede_tdesc', 'zona_tdesc', 'area_tdesc',
       'jorn_tdesc', 'sexo_tdesc', 'tcol_tdesc', 'cant_dias_matr',
       'parentesco_fiador_new', 'cant_periodos_postulaciones',
       'sin_beneficios', 'cae', 'promedio_asistencia', 'promedio_notas',
       'responde_encuesta_an', 'codigo_asignatura_mat_alg_ss', 'es_desertor',
       'ev_diagnostica_2_new', 'ev_diagnostica_4_new', 'p_asig_riesgo_new',
       'veces_accesos_app_new', 'monto_deuda_new'],
      dtype='object')

# DUMMIES

In [24]:
# Genero Dummies de las variables categoricas 
df_data = pd.get_dummies(df_data,drop_first="True")

In [25]:
df_data.shape

(125993, 95)

## Separación en train y test para generar archivos que se trabajarn en los modelos 

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
# Realizo una primera separación para dejar datos exlcusivamente para validar 
train, test= train_test_split(df_data,test_size=0.2,  random_state = 0, stratify=df_data['es_desertor'])
#

In [28]:
y_train = train.pop('es_desertor')
X_train = train

In [29]:
y_test= test.pop('es_desertor')
X_test = test

In [31]:
X_test.reset_index().to_feather('../data/processed/X_test.feather')
X_train.reset_index().to_feather('../data/processed/X_train.feather')

y_test.reset_index().to_feather('../data/processed/y_test.feather')
y_train.reset_index().to_feather('../data/processed/y_train.feather')

df_data.reset_index().to_feather('../data/raw/data.feather')